In [1]:
import gradio as gr
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

def make_rhs_func(expr_str):
    allowed_names = {
        'torch': torch,
        'sin': torch.sin, 'cos': torch.cos, 'exp': torch.exp, 'log': torch.log,
        'tan': torch.tan, 'abs': torch.abs, 'sqrt': torch.sqrt,
        'pi': torch.pi, 'e': np.e
    }
    code = compile(expr_str, "<string>", "eval")
    def rhs_func(x, y):
        env = dict(allowed_names)
        env.update({'x': x, 'y': y})
        return eval(code, {"__builtins__": None}, env)
    return rhs_func

class NeuralNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(1, 50),
            nn.Tanh(),
            nn.Linear(50, 50),
            nn.Tanh(),
            nn.Linear(50, 1)
        )
    def forward(self, x):
        return self.net(x)

def solve_ode(rhs_expr, x0_val, y0_val, epochs, x_min, x_max):
    device = torch.device("cpu")
    torch.manual_seed(42)
    # Handle bad domain (swap if min > max)
    if x_min > x_max:
        x_min, x_max = x_max, x_min
    x = torch.arange(x_min, x_max, 0.05, requires_grad=True, device=device).unsqueeze(1)
    x0 = torch.tensor([[x0_val]], dtype=torch.float32, device=device, requires_grad=True)
    y0 = torch.tensor([[y0_val]], dtype=torch.float32, device=device, requires_grad=True)
    rhs_func = make_rhs_func(rhs_expr)
    model = NeuralNet().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    loss_fn = nn.MSELoss()
    for epoch in range(epochs):
        optimizer.zero_grad()
        y_pred = model(x)
        D = torch.autograd.grad(y_pred, x, grad_outputs=torch.ones_like(y_pred), create_graph=True)[0]
        lhs = D
        rhs = rhs_func(x, y_pred)
        loss_eq = loss_fn(lhs, rhs)
        y0_pred = model(x0)
        loss_ic = loss_fn(y0_pred, y0)
        loss = loss_eq + loss_ic
        loss.backward()
        optimizer.step()
    with torch.no_grad():
        y_sol = model(x).cpu().numpy().flatten()
        x_plot = x.cpu().numpy().flatten()
        y_true = x_plot**5 / 5
    fig, ax = plt.subplots(figsize=(8,4))
    ax.plot(x_plot, y_sol, label="NN Solution")
    if rhs_expr.strip().replace(' ', '') == "x**4" and x0_val == 0 and y0_val == 0:
        ax.plot(x_plot, y_true, '--', label="Analytical: $x^5/5$")
    ax.set_xlabel('x')
    ax.set_ylabel('y(x)')
    ax.set_title(f"Solution domain: [{x_min}, {x_max}]")
    ax.legend()
    ax.grid(True)
    return fig

iface = gr.Interface(
    fn=solve_ode,
    inputs=[
        gr.Textbox(label="dy/dx =", value="x**4"),
        gr.Number(label="x0", value=0.0),
        gr.Number(label="y0", value=0.0),
        gr.Slider(100, 10000, value=1000, step=100, label="Epochs"),
        gr.Number(label="x_min", value=-2.0),
        gr.Number(label="x_max", value=2.0),
    ],
    outputs=gr.Plot(label="Solution Plot"),
    title="Neural ODE Solver (PINN) with Custom Domain"
)

iface.launch()


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
